![webinar-Banner](https://github.com/TBFY/general/blob/master/figures/webinar_banner.png?raw=true)

**INTRODUCCIÓN**

Este notebook ha sido creado como apoyo para el taller "Reutilización de datos abiertos: el caso de la contratación pública", impartido el 18 de noviembre de 2020 en el marco de la jornada formativa [Más de 25 años de Web Municipal](https://www.zaragoza.es/sede/portal/participacion/25-web-municipal).

Este notebook tiene licencia [CC BY-NC-SA 4.0 Internacional](https://creativecommons.org/licenses/by-nc-sa/4.0/). 

**Objetivos**

1. Conocer los conjuntos de [datos abiertos](https://www.zaragoza.es/sede/portal/datos-abiertos/servicio/catalogo) relacionados con la contratación pública en el [Ayuntamiento de Zaragoza](https://www.zaragoza.es/sede/).
1. Acceder a la [API REST de datos abiertos](https://www.zaragoza.es/docs-api_sede/) y realizar consultas de prueba de los diferentes conjuntos de datos.

**Metodología**

Para llevar a cabo estas consultas, se realizarán una serie de ejercicios que englobarán la mayoría de los recursos proporcionados por la API, dedicados a la contratación pública, y se dividirán en dos bloques:

1. Consultas sobre contratos
1. Consultas sobre licitaciones

Cada bloque está compuesto de una serie de preguntas que deberán ser resueltas a través de los formularios creados


In [ ]:
import requests
import pandas as pd
from IPython.display import display, HTML

 **Bloque 1- RECURSO CONTRATOS**

En este bloque se va a responder una serie de cuestiones para explotar el recurso [GET /servicio/contratacion-publica/contrato](https://www.zaragoza.es/docs-api_sede/#!/Ayuntamiento%3A_Contrataci%C3%B3n_p%C3%BAblica/get_servicio_contratacion_publica_contrato). Introduzca en el formulario de debajo los datos necesarios para resolver cada una de ellas.

1.       Obtener todos los contratos adjudicados por el Ayuntamiento de Zaragoza
2.       Obtener los contratos que tiene en fase de licitación la entidad Zaragoza Vivienda
3.       Obtener los contratos de Servicios licitados en 2017 por la entidad Zaragoza Cultural
4.       Obtener las obras licitadas por el Ayuntamiento con un importe de licitación sin IVA superior a  50.000 euros
5. Obtener los contratos en los que ha licitado El Corte Inglés (opción A)

Nota: El identificador de licitador de El Corte Inglés es 366

In [ ]:
#@title Recurso Contratos

max_number = 12 #@param {type:"slider", min:1, max:100, step:1}

entidad = "" #@param ["", "Ayuntamiento de Zaragoza", "Educación y Bibliotecas", "Zaragoza Cultural", "Zaragoza Vivienda", "Zaragoza Deporte"] {type:"string"}   

tipo_contrato = "Suministros" #@param ["", "Obras", "Suministros", "Servicios"] {type:"string"}   

status = "Pendiente de Adjudicar" #@param ["", "En licitación", "Pendiente de Adjudicar", "Adjudicación", "Adjudicación Definitiva", "Desierto", "Contrato Formalizado ", "Cancelado"] {type:"string"}  

año = "" #@param ["", "2016", "2017", "2018", "2019", "2020"] {type:"string"}  

licitador = 366 #@param {type:"integer"}

importe_sin_IVA = 0 #@param {type:"integer"}

def reemplazar (texto):
  texto = texto.replace(" ", "%20")
  texto = texto.replace("ó", "%C3%B3")
  return texto

ent = ""
cont = ""
stat = ""
date = ""
imp = ""
lic = ""

if len(entidad) >0:
  ent = '&entity.title=' + reemplazar(entidad)

if len(tipo_contrato) >0:
  cont = '&type.title=' + reemplazar(tipo_contrato)

if len(status) >0:
  stat = '&status.title=' + reemplazar(status)

if importe_sin_IVA > 0:
  imp = '&q=importeSinIVA%3Dge%3D' + str(importe_sin_IVA)

if licitador > 0:
  lic = '&q=fechaContrato%3Dgt%3D01-01-2008%2000%3A00&licitador=' + str(licitador)

if len(año) >0:
  date = '&q=fechaContrato%3Dge%3D01-01-' + año + ';fechaContrato%3Dle%3D31-12-' + año 

params = 'https://www.zaragoza.es/sede/servicio/contratacion-publica/contrato.json?rf=html&start=0&rows=' + str(max_number) + ent + cont + stat + date + imp + lic
print()
print('La consulta solicitada corresponde a la llamada:')
print ('Requesting : ' + params )
print()

resp = requests.get(params)
total_registros = resp.json()['totalCount']

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

print('El número total de registros obtenidos es: ') 
print(total_registros)

if (resp.json()['totalCount']) > 0:
  df = pd.DataFrame(columns=['id', 'title', 'expediente', 'tipo de contrato', 'procedimiento', 'entidad', 'status', 'fecha de contrato', 'importe sin IVA'])
  i=0
  for todo_item in resp.json()['result']:
    try:
      df.loc[i] = [todo_item['id'],todo_item['title'],todo_item['expediente'],todo_item['type']['title'],todo_item['procedimiento']['nombre'],todo_item['entity']['title'],todo_item['status']['title'],todo_item['fechaContrato'],todo_item['importeSinIVA']]
      i+=1
    except:
      df.loc[i] = [todo_item['id'],todo_item['title'],todo_item['expediente'],todo_item['type']['title'],todo_item['procedimiento']['nombre'],todo_item['entity']['title'],todo_item['status']['title'],todo_item['fechaContrato'],""]
      i+=1


display(HTML(df.to_html(justify='left')))




La consulta solicitada corresponde a la llamada:
Requesting : https://www.zaragoza.es/sede/servicio/contratacion-publica/contrato.json?rf=html&start=0&rows=12&type.title=Suministros&status.title=Pendiente%20de%20Adjudicar&q=fechaContrato%3Dgt%3D01-01-2008%2000%3A00&licitador=366

El número total de registros obtenidos es: 
0


,id,title,expediente,tipo de contrato,procedimiento,entidad,status,fecha de contrato,importe sin IVA
0,4655,SUMINISTRO ESTANTERIAS METALICAS PARA ALMACENAJE,296668-20,Suministros,Menor,Ayuntamiento de Zaragoza,Contrato Formalizado,2020-10-16T00:00:00,15000
1,4338,"CONTRATO DERIVADO DEL ACUERDO MARCO DE SUMINISTRO DE VESTUARIO PARA EL PERSONAL MUNICIPAL, (SUMINISTRO VESTUARIO PARA EL PERSONAL MUNICIPAL 2019), LOTES: 1, 2, 3, 4, 6, 7, 8, 9, 12, 13, 18, 19, 20, 21, 22, 23 Y 24",1152332-19,Suministros,Basado en Acuerdo Marco,Ayuntamiento de Zaragoza,Adjudicación,2019-12-27T00:00:00,
2,4091,"ACUERDO MARCO PARA EL SUMINISTRO, COLOCACIÓN Y LIMPIEZA DE SEÑALES (PLACAS CERÁMICA Y CARTELES METÁLICOS) Y SOPORTES (POSTES METÁLICOS) PARA LA ROTULACIÓN DE VÍAS Y FINCAS.",0850918-18,Suministros,Abierto,Ayuntamiento de Zaragoza,Contrato Formalizado,2019-06-24T00:00:00,282092
3,3708,SUMINISTRO Y COLOCACION PLACAS CERAMICAS ADOSADAS A EDIFICIOS PARA LA ROTULACION DE VIAS Y FINCAS.,202567-19,Suministros,Menor,Ayuntamiento de Zaragoza,Contrato Formalizado,2019-05-08T14:00:32,15000
4,3591,"ACUERDO MARCO DE SUMINISTRO DE VESTUARIO PARA EL PERSONAL MUNICIPAL, (SUMINISTRO VESTUARIO PARA EL PERSONAL MUNICIPAL 2018), LOTES: 1, 4, 6, 7, 8, 9, 12, 13, 15,17, 18, 19, 20, 21, 22, 23 Y 24",1445182-18,Suministros,Basado en Acuerdo Marco,Ayuntamiento de Zaragoza,Adjudicación,2018-12-28T12:24:52,
5,3591,"ACUERDO MARCO DE SUMINISTRO DE VESTUARIO PARA EL PERSONAL MUNICIPAL, (SUMINISTRO VESTUARIO PARA EL PERSONAL MUNICIPAL 2018), LOTES: 1, 4, 6, 7, 8, 9, 12, 13, 15,17, 18, 19, 20, 21, 22, 23 Y 24",1445182-18,Suministros,Basado en Acuerdo Marco,Ayuntamiento de Zaragoza,Adjudicación,2018-12-28T12:24:52,
6,3591,"ACUERDO MARCO DE SUMINISTRO DE VESTUARIO PARA EL PERSONAL MUNICIPAL, (SUMINISTRO VESTUARIO PARA EL PERSONAL MUNICIPAL 2018), LOTES: 1, 4, 6, 7, 8, 9, 12, 13, 15,17, 18, 19, 20, 21, 22, 23 Y 24",1445182-18,Suministros,Basado en Acuerdo Marco,Ayuntamiento de Zaragoza,Adjudicación,2018-12-28T12:24:52,
7,3591,"ACUERDO MARCO DE SUMINISTRO DE VESTUARIO PARA EL PERSONAL MUNICIPAL, (SUMINISTRO VESTUARIO PARA EL PERSONAL MUNICIPAL 2018), LOTES: 1, 4, 6, 7, 8, 9, 12, 13, 15,17, 18, 19, 20, 21, 22, 23 Y 24",1445182-18,Suministros,Basado en Acuerdo Marco,Ayuntamiento de Zaragoza,Adjudicación,2018-12-28T12:24:52,
8,3591,"ACUERDO MARCO DE SUMINISTRO DE VESTUARIO PARA EL PERSONAL MUNICIPAL, (SUMINISTRO VESTUARIO PARA EL PERSONAL MUNICIPAL 2018), LOTES: 1, 4, 6, 7, 8, 9, 12, 13, 15,17, 18, 19, 20, 21, 22, 23 Y 24",1445182-18,Suministros,Basado en Acuerdo Marco,Ayuntamiento de Zaragoza,Adjudicación,2018-12-28T12:24:52,
9,3591,"ACUERDO MARCO DE SUMINISTRO DE VESTUARIO PARA EL PERSONAL MUNICIPAL, (SUMINISTRO VESTUARIO PARA EL PERSONAL MUNICIPAL 2018), LOTES: 1, 4, 6, 7, 8, 9, 12, 13, 15,17, 18, 19, 20, 21, 22, 23 Y 24",1445182-18,Suministros,Basado en Acuerdo Marco,Ayuntamiento de Zaragoza,Adjudicación,2018-12-28T12:24:52,


**Bloque 2 - RECURSO LICITACIONES**

En este bloque se va a responder una serie de cuestiones para explotar el recurso [GET /servicio/contratacion-publica/licitador/{id}](https://www.zaragoza.es/docs-api_sede/#!/Ayuntamiento%3A_Contrataci%C3%B3n_p%C3%BAblica/get_servicio_contratacion_publica_licitador_id). Introduzca en el formulario de debajo los datos necesarios para resolver cada una de ellas.

5. Obtener los contratos en los que ha licitado El Corte Inglés (opción B)
6. Obtener los contratos en los que ha licitado XXX por medio de una UTE

In [ ]:
#@title Licitaciones

max_number = 100 #@param {type:"slider", min:1, max:100, step:1}

licitador = 2 #@param {type:"integer"}

tiene_UTE = True #@param {type:"boolean"}

ute = ""

if tiene_UTE:
  ute = '&tieneUte=true'

params = 'https://www.zaragoza.es/sede/servicio/contratacion-publica/licitador/' + str(licitador) + '/licitaciones.json?rf=html&start=0&rows=' + str(max_number) + ute
print()
print('La consulta solicitada corresponde a la llamada:')
print ('Requesting : ' + params )
print()
resp = requests.get(params)
total_registros = resp.json()['totalCount']

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

print('El número total de registros obtenidos es: ') 
print(total_registros)


df = pd.DataFrame(columns=['id', 'tituloContrato'])
i=0

if (resp.json()['totalCount']) > 0:
  for todo_item in resp.json()['result']:
    try:
      df.loc[i] = [todo_item['id'],todo_item['tituloContato']]
      i+=1
    except:
      df.loc[i] = [todo_item['id'],todo_item['tituloContato']]
      i+=1

display(HTML(df.to_html(justify='left')))


La consulta solicitada corresponde a la llamada:
Requesting : https://www.zaragoza.es/sede/servicio/contratacion-publica/licitador/2/licitaciones.json?rf=html&start=0&rows=100&tieneUte=true

El número total de registros obtenidos es: 
0


,id,tituloContrato


**Buscador de licitadores**

In [ ]:
#@title Listado de licitadores

max_number = 3000 #@param {type:"slider", min:1, max:3000, step:1}

nombre = "CORTE" #@param {type:"string"}


params = 'https://www.zaragoza.es/sede/servicio/contratacion-publica/licitador.json?rf=html&start=0&rows=' + str(max_number) 
print()
print('La consulta solicitada corresponde a la llamada:')
print ('Requesting : ' + params )
print()
resp = requests.get(params)

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))


df = pd.DataFrame(columns=['idEmpresa', 'nombre'])
i=0

if (resp.json()['totalCount']) > 0:
  for todo_item in resp.json()['result']:
    if (todo_item['nombre'].find(nombre)) > 0:
      try:
        df.loc[i] = [todo_item['idEmpresa'],todo_item['nombre']]
        i+=1
      except:
        df.loc[i] = [todo_item['idEmpresa'],todo_item['nombre']]
        i+=1

display(HTML(df.to_html(justify='left')))


La consulta solicitada corresponde a la llamada:
Requesting : https://www.zaragoza.es/sede/servicio/contratacion-publica/licitador.json?rf=html&start=0&rows=3000



,idEmpresa,nombre
0,175,INFORMATICA EL CORTE INGLES S.A.
